In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
import librosa
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from skimage import filters
from tensorflow.keras import regularizers
from tensorflow.keras.applications import EfficientNetV2B3, EfficientNetV2M
from tensorflow.keras.layers import (
    BatchNormalization,
    Conv2D,
    Dense,
    Flatten,
    Input,
    Lambda,
    MaxPooling2D,
)
from tensorflow.keras.models import Model
import yaml
from tensorflow.keras.models import load_model
import csv
from scipy.ndimage import gaussian_filter
from scipy.ndimage import median_filter

In [ ]:
class TransferLearningModelBuilder:
    def __init__(self, model_params):
        self.input_shape = model_params["input_shape"]
        self.learning_rate = model_params.get("learning_rate", 0.001)
        self.l1 = model_params.get("l1", 0.0)
        self.base_model_name = model_params.get("base_model_name", "EfficientNetV2B3")
        self.weight_initialization = model_params.get("weight_initialization", "he_normal")
        self.last_layers_to_train = model_params.get("last_layers_to_train", 0)
        self.dropout = model_params.get("dropout", 0.0)
        self.num_classes = model_params.get("num_classes", 1)  # Number of output classes
        self.model = None
        self.base_model = None

    def build_base_model(self):
        # Input layer for the grayscale image
        input_layer = tf.keras.layers.Input(shape=self.input_shape)

        # Check if the image is grayscale (channel dimension is 1)
        if self.input_shape[-1] == 1:
            # Lambda layer to repeat the grayscale channel three times
            x = Lambda(lambda x: tf.repeat(x, 3, axis=-1))(input_layer)
        else:
            x = input_layer

        # Base model initialization
        if self.base_model_name == "EfficientNetV2B3":
            self.base_model = EfficientNetV2B3(weights="imagenet", include_top=False, input_tensor=x)
        elif self.base_model_name == "EfficientNetV2L":
            self.base_model = EfficientNetV2M(weights="imagenet", include_top=False, input_tensor=x)
        else:
            raise ValueError("Invalid base model name")

        # Freeze all layers of base model for transfer learning
        for layer in self.base_model.layers[: -self.last_layers_to_train]:
            layer.trainable = False

        return Model(inputs=input_layer, outputs=self.base_model.output)

    def build(self):
        base_model = self.build_base_model()
        x = Flatten()(base_model.output)  # Flatten the output to connect with Dense layer

        # Classifier with L1 regularization
        if self.dropout > 0.0:
            x = tf.keras.layers.Dropout(self.dropout)(x)
        output = Dense(
            21,  # Change to number of classes for multi-class classification
            activation="softmax",  # Use softmax for multi-class classification
            kernel_regularizer=regularizers.l1(self.l1),
            kernel_initializer=self.weight_initialization,
        )(x)

        self.model = Model(inputs=base_model.input, outputs=output)

    def compile(self):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate),
            loss="categorical_crossentropy",  # Use categorical_crossentropy for multi-class classification
            metrics=["accuracy", tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), tf.keras.metrics.AUC(name='auc')],
        )
        return self.model

In [ ]:
# Load YAML file
with open('config.yaml', 'r') as file:
    config = yaml.safe_load(file)

In [ ]:
# Create TransferLearningModelBuilder instance with parameters from YAML file
model_builder = TransferLearningModelBuilder(config['model_params'])

In [ ]:
# Build and compile the model
model_builder.build()
model = model_builder.compile()

In [ ]:
# Directories for training and validation data
train_dir = 'train_directory'
val_dir = 'val_directory'

# Create ImageDataGenerator for training and validation
train_datagen = ImageDataGenerator()
val_datagen = ImageDataGenerator()

# Create generators for training and validation
train_generator = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)

val_generator = val_datagen.flow_from_directory(
    directory=val_dir,
    target_size=(128, 128),
    batch_size=64,
    class_mode='categorical'  # Use 'categorical' for multi-class classification
)



In [ ]:
# Define a callback to save the model at each epoch
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.keras',
    save_best_only=True,
    monitor='val_loss',
    mode='min',
    verbose=1
)

# Implement early stopping to avoid overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
history = model.fit(train_generator, validation_data=val_generator, epochs=10, callbacks=[checkpoint_callback, early_stopping])